In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset_dir = "./Plant_Village_Data_Set/"

In [3]:
dataset_dir

'./Plant_Village_Data_Set/'

In [4]:
from sklearn.model_selection import train_test_split
import os

# List all subdirectories (each corresponds to a disease category)
subdirectories = [subdir for subdir in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, subdir))]

# Create lists to store image paths and corresponding labels
image_paths = []
labels = []
sum = 0
# Iterate through each subdirectory (disease category)
for subdir in subdirectories:
    subdir_path = os.path.join(dataset_dir, subdir)
    
    subdir_images = [os.path.join(subdir_path, img) for img in os.listdir(subdir_path) if img.lower().endswith('.jpg')]

    sum += (len(subdir_images))
    image_paths.extend(subdir_images)
    labels.extend([subdir] * len(subdir_images))
print(sum)
# Split the data into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


20636


In [5]:
subdirectories

['Tomato_healthy',
 'Potato___Early_blight',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato_Early_blight',
 'Tomato__Target_Spot',
 'Potato___Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato_Septoria_leaf_spot',
 'Tomato__Tomato_mosaic_virus',
 'Pepper__bell___Bacterial_spot',
 'Tomato_Bacterial_spot',
 'Tomato_Late_blight',
 'Pepper__bell___healthy',
 'Potato___healthy']

In [6]:
test_labels

['Potato___Late_blight',
 'Potato___Early_blight',
 'Tomato_Bacterial_spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Bacterial_spot',
 'Tomato_Leaf_Mold',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Early_blight',
 'Tomato__Target_Spot',
 'Tomato_Late_blight',
 'Tomato_Bacterial_spot',
 'Tomato_Septoria_leaf_spot',
 'Potato___Late_blight',
 'Potato___Early_blight',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Pepper__bell___healthy',
 'Tomato_healthy',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato__Target_Spot',
 'Tomato_Bacterial_spot',
 'Tomato_Bacterial_spot',
 'Tomato_Late_blight',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Potato___Early_blight',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Leaf_Mold',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__To

In [7]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
def load_images(image_paths):
    images = []
    for path in image_paths:
        image = cv2.imread(path)  # Load image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        images.append(image)
    return np.array(images)

# Load training and testing images
train_images = load_images(train_paths)
test_images = load_images(test_paths)

# Convert labels to categorical format (if using one-hot encoding)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

num_classes = len(np.unique(train_labels_encoded))
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
test_labels_onehot = to_categorical(test_labels_encoded, num_classes=num_classes)


In [56]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical

# Preprocess images (normalize pixel values and resize)
def preprocess_images(images, target_size=(224, 224)):
    processed_images = []
    for img in images:
        img = img.astype('float32') / 255.0  # Normalize pixel values
        img = cv2.resize(img, target_size)   # Resize image
        processed_images.append(img)
    return np.array(processed_images)

# Preprocess training and testing images
train_images_processed = preprocess_images(train_images)
test_images_processed = preprocess_images(test_images)

# Check shapes of preprocessed data
print("Train Images Shape:", train_images_processed.shape)
print("Test Images Shape:", test_images_processed.shape)
print("Train Labels Shape:", train_labels_onehot.shape)
print("Test Labels Shape:", test_labels_onehot.shape)


Train Images Shape: (16508, 224, 224, 3)
Test Images Shape: (4128, 224, 224, 3)
Train Labels Shape: (16508, 15)
Test Labels Shape: (4128, 15)


In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images_processed, train_labels_onehot,
                    batch_size=32, epochs=10, validation_split=0.2)


/Users/pavan/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 1373s 3s/step - accuracy: 0.3702 - loss: 1.9876 - val_accuracy: 0.7813 - val_loss: 0.7083
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 316s 763ms/step - accuracy: 0.7218 - loss: 0.8414 - val_accuracy: 0.8110 - val_loss: 0.5592
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 352s 852ms/step - accuracy: 0.8115 - loss: 0.5715 - val_accuracy: 0.8719 - val_loss: 0.3698
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 391s 946ms/step - accuracy: 0.8565 - loss: 0.4197 - val_accuracy: 0.8683 - val_loss: 0.3780
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 412s 996ms/step - accuracy: 0.8929 - loss: 0.3167 - val_accuracy: 0.8985 - val_loss: 0.2941
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 314s 760ms/step - accuracy: 0.9219 - loss: 0.2339 - val_accuracy: 0.8949 - val_loss: 0.3227
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 311s 753ms/step - accuracy: 0.9276 - loss: 0.2220 - val_accuracy: 0.8740 - val_loss: 0.4103
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 335s 811ms/step - accuracy: 0.9492 - l

In [8]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_images_processed, test_labels_onehot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


NameError: name 'model' is not defined

In [59]:
# Make predictions on test data
predictions = model.predict(test_images_processed)

import numpy as np

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(test_labels_encoded, predicted_labels)

# Display the confusion matrix
classes = label_encoder.classes_
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

# Plot confusion matrix



129/129 ━━━━━━━━━━━━━━━━━━━━ 27s 211ms/step


In [60]:
predicted_labels

array([ 3,  2,  5, ..., 11,  5,  1])

In [63]:
cm

array([[180,   7,   8,   1,   0,   0,   0,   6,   0,  12,   0,   0,   0,
          0,   0],
       [  3, 283,   0,   1,   1,   0,   1,   0,   1,   2,   0,   0,   0,
          0,   0],
       [  0,   0, 180,   7,   0,   0,   0,   2,   0,   3,   0,   0,   0,
          0,   0],
       [  1,   0,   5, 188,   0,   0,   0,   7,   0,   2,   0,   2,   0,
          0,   0],
       [  0,   9,   0,   6,  14,   0,   0,   0,   0,   2,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   2,   0, 410,   3,  10,   0,   3,   0,   2,   9,
          0,   0],
       [  1,   1,   2,   2,   0,   4, 140,  25,   3,  10,   0,   9,   2,
          0,   0],
       [  0,   0,   2,  19,   0,   1,  14, 329,   4,   8,   0,   1,   2,
          0,   0],
       [  1,   1,   0,   0,   0,   0,   2,   6, 155,  22,   0,   1,   0,
          0,   1],
       [  3,   3,   2,   0,   0,   1,   2,   4,   6, 339,   0,   1,   0,
          2,   0],
       [  0,   0,   0,   0,   0,   0,   1,   1,   1,   1, 288,  19,   0,
       